# PythonによるLIMEの実装例

## データの読み込み

In [ ]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

dataset = fetch_california_housing()
X_tr, X_te, y_tr, y_te = train_test_split(
    dataset.data, dataset.target, test_size=0.2)
X_scaler = StandardScaler().fit(X_tr)
X_tr = X_scaler.transform(X_tr)
X_te = X_scaler.transform(X_te)

In [ ]:
import pandas as pd
print(pd.DataFrame(X_tr, columns=dataset.feature_names))

In [ ]:
print(pd.DataFrame(y_tr))

## 予測モデルの学習

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

pred_model = MLPRegressor(hidden_layer_sizes=(100,100,))
pred_model.fit(X_tr, y_tr)
y_pred = pred_model.predict(X_te)
mean_squared_error(y_te, y_pred)

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline
plt.scatter(y_te, y_pred)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()

## 疑似事例の生成器クラスの定義

In [ ]:
import scipy

class SampleGenerator(object):
    def prepare(self, X_tr):
        _, d = X_tr.shape
        self.bins = []
        self.means = []
        self.stds = []
        self.mins = []
        self.maxs = []
        for j in range(d):
            '''四分位数を求める'''
            qts = np.percentile(X_tr[:, j], [25, 50, 75])
            self.bins.append(qts)

            '''どの四分位に所属するのかを求める'''
            bidxes = np.digitize(X_tr[:, j], qts)

            '''各四分位の平均・標準偏差・最小・最大を求める'''
            means_k = []
            stds_k = []
            for k in range(len(qts) + 1):
                X_selected = X_tr[bidxes == k, j]
                means_k.append(X_selected.mean())
                stds_k.append(X_selected.std())
            self.means.append(means_k)
            self.stds.append(stds_k)
            self.mins.append(
                [X_tr[bidxes == 0, j].min()] + qts.tolist())
            self.maxs.append(
                qts.tolist() + [X_tr[bidxes == len(qts), j].max()])

    def generate(self, x, n=100):
        d = X_tr.shape[1]

        '''テスト事例の各特徴量が所属する四分位を求める'''
        bi_true = []
        for j in range(d):
            bi_true.append(np.digitize(x[j], self.bins[j]))
        bi_true = np.array(bi_true)
        
        '''疑似事例の元表現と解釈表現を生成する'''
        bi_gen = np.random.randint(0, 4, size=(n, d))
        Z_orig = np.zeros((n, d))
        Z_interpret = np.zeros((n, d))
        for i in range(n):
            Z_interpret[i] = np.where(bi_gen[i] == bi_true, 1, 0)
            for j in range(d):
                bi = bi_gen[i, j]
                min_norm = (self.mins[j][bi] - self.means[j][bi]) / self.stds[j][bi]
                max_norm = (self.maxs[j][bi] - self.means[j][bi]) / self.stds[j][bi]
                Z_orig[i, j] = scipy.stats.truncnorm.rvs(
                    min_norm, max_norm, 
                    loc=self.means[j][bi], scale=self.stds[j][bi])
        
        return Z_orig, Z_interpret


In [ ]:
gen = SampleGenerator()
gen.prepare(X_tr)
Z_orig, Z_interpret = gen.generate(X_te[0], n=5)
print(pd.DataFrame(Z_orig, columns=dataset.feature_names))
print(pd.DataFrame(Z_interpret, columns=dataset.feature_names))

## LIMEによる説明（重み付き線形回帰モデルの学習）

In [ ]:
from sklearn.linear_model import Ridge

gen = SampleGenerator()
gen.prepare(X_tr)
Z_orig, Z_interpret = gen.generate(X_te[0], n=100)
kernels = np.exp(-np.linalg.norm(X_te[0] - Z_orig, axis=1)**2 / 2.)
y_pred = pred_model.predict(Z_orig)
exp = Ridge(fit_intercept=False).fit(
    Z_interpret, y_pred, sample_weight=kernels)

print(pd.DataFrame(exp.coef_, index=dataset.feature_names))
